In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import layers

import pennylane as qml
import numpy as np

import matplotlib.pyplot as plt
import strawberryfields as sf

In [2]:
mnist = keras.datasets.mnist

# datasets are numpy.ndarrays
(X_train, Y_train),  (X_test, Y_test) = mnist.load_data()


In [3]:
# normalize the pixels from 0 ~ 225 to 0 ~ 1
X_train, X_test = X_train / 255.0, X_test / 255.0

In [4]:
def one_hot(labels):

    depth =  4**2                       # 10 classes + 6 zeros for padding
    indices = labels.astype(np.int32)
    one_hot_labels = np.eye(depth)[indices].astype(np.float32)

    return one_hot_labels

# one-hot encoded labels, each label of length cutoff dimension**2
y_train,y_test = one_hot(Y_train), one_hot(Y_test)

# using only 700 samples for training in this experiment
n_samples = 500
test_samples = 100
X_train, X_val, X_test, y_train,y_val, y_test = X_train[:n_samples], X_train[n_samples:700],X_test[:test_samples], y_train[:n_samples],y_train[n_samples:700], y_test[:test_samples]

In [20]:
keras.backend.set_floatx('float64')

model = keras.models.Sequential([
                                 layers.Flatten(input_shape = (28,28)),
                                 layers.Dense(128, activation ="elu"),
                                 layers.Dense(64, activation ="elu"),
                                 layers.Dense(32, activation ="elu"),
                                 layers.Dense(22, activation ="elu")
                                ])

# More than a million parameters for the classical circuit
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_4 (Dense)             (None, 128)               100480    
                                                                 
 dense_5 (Dense)             (None, 64)                8256      
                                                                 
 dense_6 (Dense)             (None, 32)                2080      
                                                                 
 dense_7 (Dense)             (None, 22)                726       
                                                                 
Total params: 111,542
Trainable params: 111,542
Non-trainable params: 0
_________________________________________________________________


In [30]:
def init_layer(x):
    qml.Squeezing(x[0], 0.0, wires=0)
    qml.Squeezing(x[1], 0.0, wires=1)
    qml.Squeezing(x[2], 0.0, wires=2)

    qml.Beamsplitter(x[3], x[4], wires=[0,1])
    qml.Beamsplitter(x[5], x[6], wires=[1,2])

    qml.Rotation(x[7], wires=0)
    qml.Rotation(x[8], wires=1)
    qml.Rotation(x[9], wires=2)

    qml.Displacement(x[10], 0.0, wires=0)
    qml.Displacement(x[11], 0.0, wires=1)
    qml.Displacement(x[12], 0.0, wires=2)

    qml.Kerr(x[13], wires=0)
    qml.Kerr(x[14], wires=1)
    qml.Kerr(x[15], wires=2)

In [31]:
def layer(v):
    qml.Beamsplitter(v[0], v[1], wires=[0,1])
    qml.Beamsplitter(v[2], v[3], wires=[1,2])

    qml.Rotation(v[4], wires=0)
    qml.Rotation(v[5], wires=1)
    qml.Rotation(v[6], wires=2)

    qml.Squeezing(v[7], 0.0, wires=0)
    qml.Squeezing(v[8], 0.0, wires=1)
    qml.Squeezing(v[9], 0.0, wires=2)

    qml.Beamsplitter(v[10], v[11], wires=[0,1])
    qml.Beamsplitter(v[12], v[13], wires=[1,2])

    qml.Rotation(v[14], wires=0)
    qml.Rotation(v[15], wires=1)
    qml.Rotation(v[16], wires=2)

    qml.Displacement(v[17], 0.0, wires=0)
    qml.Displacement(v[18], 0.0, wires=1)
    qml.Displacement(v[19], 0.0, wires=2)

    qml.Kerr(v[20], wires=0)
    qml.Kerr(v[21], wires=1)
    qml.Kerr(v[22], wires=2)

In [32]:
def init_weights(layers, modes, active_sd=0.0001, passive_sd=0.1):

    M = 4+3  # Number of interferometer parameters: 2 2-parameter beamsplitters + 3 rotations

    int1_weights = tf.random.normal(shape=[layers, M], stddev=passive_sd)
    s_weights = tf.random.normal(shape=[layers, modes], stddev=active_sd)
    int2_weights = tf.random.normal(shape=[layers, M], stddev=passive_sd)
    dr_weights = tf.random.normal(shape=[layers, modes], stddev=active_sd)
    k_weights = tf.random.normal(shape=[layers, modes], stddev=active_sd)

    weights = tf.concat([int1_weights, s_weights, int2_weights, dr_weights, k_weights], axis=1)
    weights = tf.Variable(weights)

    return weights

In [33]:
num_modes = 3
num_basis = 3

dev = qml.device("strawberryfields.fock", wires=num_modes, cutoff_dim=num_basis)

@qml.qnode(dev, interface="tf")
def quantum_nn(inputs, var):
    init_layer(inputs)            # Encode input x into quantum state

    for v in var:                 # iterative quantum layers
        layer(v)

    return qml.probs(wires=[0, 1, 2])  # Measurement

In [34]:
num_layers = 4

# initialize weights for quantum layers
weights = init_weights(num_layers, num_modes)

# Convert the quantum layer to a Keras layer
shape_tup = weights.shape
weight_shapes = {'var': shape_tup}
qlayer = qml.qnn.KerasLayer(quantum_nn, weight_shapes, output_dim = 4)

# Add to the classical sequential model
model.add(qlayer)

In [26]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [27]:
variable_learning_rate = ReduceLROnPlateau(monitor='val_loss', factor = 0.2, patience = 2)

In [35]:
opt = keras.optimizers.SGD(lr = 0.03)
model.compile(opt, loss = 'categorical_crossentropy', metrics =['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_4 (Dense)             (None, 128)               100480    
                                                                 
 dense_5 (Dense)             (None, 64)                8256      
                                                                 
 dense_6 (Dense)             (None, 32)                2080      
                                                                 
 dense_7 (Dense)             (None, 22)                726       
                                                                 
 keras_layer_1 (KerasLayer)  (None, 4)                 92        
                                                                 
 keras_layer_2 (KerasLayer)  (None, 4)                

E:\git_repo_asus\CSE499.06(QML)\venv\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.


In [37]:
hybrid = model.fit(X_train.astype('float32'),
                   y_train.astype('float32'),
                   epochs = 100,
                   batch_size = 64,
                   shuffle = True,

                   validation_data = (X_val.astype('float32'), y_val.astype('float32'))
                   )

Epoch 1/100


ValueError: Shapes (64, 16) and (64, 27) are incompatible